## W&B Key - "4d4d4c8f520ba4f560db3da3a2e051523a812c46"

In [1]:
!nvidia-smi

Sat Jan 10 09:13:07 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   31C    P0             53W /  400W |       0MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
!git clone https://github.com/PerforatedAI/PerforatedAI.git
%cd PerforatedAI

# install
!pip install -e .

# Install dependencies
!pip install -q transformers datasets accelerate wandb

Cloning into 'PerforatedAI'...
remote: Enumerating objects: 1383, done.
remote: Counting objects: 100% (339/339), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 1383 (delta 249), reused 230 (delta 230), pack-reused 1044 (from 2)
Receiving objects: 100% (1383/1383), 72.27 MiB | 11.43 MiB/s, done.
Resolving deltas: 100% (764/764), done.
/content/PerforatedAI
Obtaining file:///content/PerforatedAI
  Preparing metadata (setup.py) ... done
  Running setup.py develop for perforatedai


## Generic Train and Eval Loader with basline and dentrics

In [3]:
import os
import math
import random
import numpy as np

import torch
from torch.utils.data import DataLoader

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    get_linear_schedule_with_warmup,
)

from perforatedai import globals_perforatedai as GPA
from perforatedai import utils_perforatedai as UPA

import wandb

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Building dendrites without Perforated Backpropagation
Using device: cuda


In [4]:
# Models
BASELINE_MODEL_NAME = "distilbert-base-uncased"      # baseline
COMPRESSED_MODEL_NAME = "prajjwal1/bert-tiny"        # smaller "compressed" model

DATASET_NAME = "glue"
DATASET_CONFIG = "sst2"  # sentiment classification
NUM_LABELS = 2

BATCH_SIZE = 32
BASELINE_EPOCHS = 3
COMPRESSED_EPOCHS = 3
INITIAL_LR = 2e-5

SEED = 42

# W&B config (optional but recommended)
WANDB_PROJECT = "perforatedai-hackathon"
os.environ["WANDB_PROJECT"] = WANDB_PROJECT

In [5]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(SEED)

In [6]:
import wandb
wandb.login(key="4d4d4c8f520ba4f560db3da3a2e051523a812c46")

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: warkevishal29 (warkevishal292) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [7]:
tokenizer = AutoTokenizer.from_pretrained(BASELINE_MODEL_NAME)

raw_datasets = load_dataset(DATASET_NAME, DATASET_CONFIG)

def preprocess_function(examples):
    return tokenizer(
        examples["sentence"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )

encoded_datasets = raw_datasets.map(preprocess_function, batched=True)

encoded_datasets = encoded_datasets.rename_column("label", "labels")
encoded_datasets.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"],
)

train_dataset = encoded_datasets["train"]
eval_dataset = encoded_datasets["validation"]

print(train_dataset[0])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

{'labels': tensor(0), 'input_ids': tensor([  101,  5342,  2047,  3595,  8496,  2013,  1996, 18643,  3197,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0, 

In [8]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
eval_dataloader = DataLoader(eval_dataset, batch_size=BATCH_SIZE)

In [9]:
def evaluate(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    loss_sum = 0.0

    with torch.no_grad():
        for batch in dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            logits = outputs.logits

            preds = torch.argmax(logits, dim=-1)
            correct += (preds == batch["labels"]).sum().item()
            total += batch["labels"].size(0)
            loss_sum += loss.item() * batch["labels"].size(0)

    avg_loss = loss_sum / total
    accuracy = correct / total
    return {"loss": avg_loss, "accuracy": accuracy}

In [10]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def benchmark_inference(model, dataloader, n_batches=10):
    model.eval()
    import time
    start = time.time()
    with torch.no_grad():
        for i, batch in enumerate(dataloader):
            if i >= n_batches:
                break
            batch = {k: v.to(device) for k, v in batch.items()}
            _ = model(**batch)
    end = time.time()
    return (end - start) / n_batches

In [11]:
baseline_model = AutoModelForSequenceClassification.from_pretrained(
    BASELINE_MODEL_NAME,
    num_labels=NUM_LABELS,
).to(device)

baseline_optimizer = torch.optim.AdamW(baseline_model.parameters(), lr=INITIAL_LR)

num_training_steps = BASELINE_EPOCHS * math.ceil(len(train_dataloader))
baseline_scheduler = get_linear_schedule_with_warmup(
    baseline_optimizer,
    num_warmup_steps=int(0.1 * num_training_steps),
    num_training_steps=num_training_steps,
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
def train_baseline(model, optimizer, scheduler, num_epochs=3, run_name="baseline-no-dendrites"):
    run = wandb.init(project=WANDB_PROJECT, name=run_name)
    global_step = 0

    for epoch in range(num_epochs):
        model.train()
        for batch in train_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

            global_step += 1
            if global_step % 50 == 0:
                wandb.log({"train/loss": loss.item(), "step": global_step})

        metrics = evaluate(model, eval_dataloader)
        print(f"[Baseline] Epoch {epoch} | val_loss={metrics['loss']:.4f} | val_acc={metrics['accuracy']:.4f}")
        wandb.log(
            {
                "val/loss": metrics["loss"],
                "val/accuracy": metrics["accuracy"],
                "epoch": epoch,
            }
        )

    run.finish()
    return model

baseline_model = train_baseline(baseline_model, baseline_optimizer, baseline_scheduler, BASELINE_EPOCHS)
baseline_metrics = evaluate(baseline_model, eval_dataloader)
print("Baseline metrics:", baseline_metrics)
print("Baseline params:", count_parameters(baseline_model))

[Baseline] Epoch 0 | val_loss=0.2427 | val_acc=0.9048
[Baseline] Epoch 1 | val_loss=0.2957 | val_acc=0.9048
[Baseline] Epoch 2 | val_loss=0.3570 | val_acc=0.9025


epoch,▁▅█
step,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇████
train/loss,█▄▅▄▅▄▃▂▃▃▅▂▄▂▃▃▃▃▁▁▁▁▃▁▂▁▃▁▄▁▁▁▂▄▂▃▂▁▁▃
val/accuracy,██▁
val/loss,▁▄█
epoch,2
step,6300
train/loss,0.03724
val/accuracy,0.90252
val/loss,0.35695


Baseline metrics: {'loss': 0.3569536072740314, 'accuracy': 0.9025229357798165}
Baseline params: 66955010


In [13]:
compressed_model = AutoModelForSequenceClassification.from_pretrained(
    COMPRESSED_MODEL_NAME,
    num_labels=NUM_LABELS,
).to(device)

compressed_optimizer = torch.optim.AdamW(compressed_model.parameters(), lr=INITIAL_LR)

num_training_steps = COMPRESSED_EPOCHS * math.ceil(len(train_dataloader))
compressed_scheduler = get_linear_schedule_with_warmup(
    compressed_optimizer,
    num_warmup_steps=int(0.1 * num_training_steps),
    num_training_steps=num_training_steps,
)

config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
def train_compressed(model, optimizer, scheduler, num_epochs=3, run_name="compressed-no-dendrites"):
    run = wandb.init(project=WANDB_PROJECT, name=run_name)
    global_step = 0

    for epoch in range(num_epochs):
        model.train()
        for batch in train_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

            global_step += 1
            if global_step % 50 == 0:
                wandb.log({"train/loss": loss.item(), "step": global_step})

        metrics = evaluate(model, eval_dataloader)
        print(f"[Compressed] Epoch {epoch} | val_loss={metrics['loss']:.4f} | val_acc={metrics['accuracy']:.4f}")
        wandb.log(
            {
                "val/loss": metrics["loss"],
                "val/accuracy": metrics["accuracy"],
                "epoch": epoch,
            }
        )

    run.finish()
    return model

compressed_model = train_compressed(compressed_model, compressed_optimizer, compressed_scheduler, COMPRESSED_EPOCHS)
compressed_metrics = evaluate(compressed_model, eval_dataloader)
print("Compressed (no dendrites) metrics:", compressed_metrics)
print("Compressed (no dendrites) params:", count_parameters(compressed_model))

model.safetensors:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

[Compressed] Epoch 0 | val_loss=0.4861 | val_acc=0.7672
[Compressed] Epoch 1 | val_loss=0.4585 | val_acc=0.7833
[Compressed] Epoch 2 | val_loss=0.4532 | val_acc=0.7913


epoch,▁▅█
step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇██
train/loss,████▇▆▅▅▃▃▅▅▆█▃▂▂▄▆▃▄▂▅▂▅▃▅▇▃▂▃▂▂▁▄▃▃▅▂▄
val/accuracy,▁▆█
val/loss,█▂▁
epoch,2
step,6300
train/loss,0.44331
val/accuracy,0.79128
val/loss,0.45318


Compressed (no dendrites) metrics: {'loss': 0.45317753793996407, 'accuracy': 0.7912844036697247}
Compressed (no dendrites) params: 4386178


In [15]:
# dentric model
from perforatedai import globals_perforatedai as GPA
from perforatedai import utils_perforatedai as UPA
import torch

GPA.pc.set_unwrapped_modules_confirmed(True)

GPA.pc.set_testing_dendrite_capacity(True)

dendritic_model = AutoModelForSequenceClassification.from_pretrained(
    COMPRESSED_MODEL_NAME,
    num_labels=NUM_LABELS,
)

dendritic_model = UPA.initialize_pai(dendritic_model)
dendritic_model.to(device)

#    Output shape: [batch, seq_len, hidden_dim]; hidden_dim is neuron axis -> 0
output_dim_vec = torch.tensor([-1, -1, 0], device=device)

for block in dendritic_model.bert.encoder.layer:
    # feed-forward blocks
    block.intermediate.dense.set_this_output_dimensions(output_dim_vec)
    block.output.dense.set_this_output_dimensions(output_dim_vec)

    # attention output projection
    block.attention.output.dense.set_this_output_dimensions(output_dim_vec)

    # attention self projections: query, key, value
    block.attention.self.query.set_this_output_dimensions(output_dim_vec)
    block.attention.self.key.set_this_output_dimensions(output_dim_vec)
    block.attention.self.value.set_this_output_dimensions(output_dim_vec)

print("Dendritic model ready. Number of params:", count_parameters(dendritic_model))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


By default skipping base_model. See "Safetensors Errors" section of customization.md to include it.
Running a test of Dendrite Capacity.
Dendritic model ready. Number of params: 4798468


In [16]:
learning_rate = INITIAL_LR

GPA.pai_tracker.set_optimizer(torch.optim.Adam)
GPA.pai_tracker.set_scheduler(torch.optim.lr_scheduler.ReduceLROnPlateau)

optim_args = {'params': dendritic_model.parameters(), 'lr': learning_rate}
sched_args = {'mode': 'max', 'patience': 3}

dend_optimizer, dend_scheduler = GPA.pai_tracker.setup_optimizer(
    dendritic_model,
    optim_args,
    sched_args,
)

print("Dendritic optimizer and scheduler initialized.")

Dendritic optimizer and scheduler initialized.


In [17]:
def train_with_dendrites(
    model,
    optimizer,
    optim_args,
    sched_args,
    max_epochs=20,
    run_name="compressed-with-dendrites"
):
    run = wandb.init(project=WANDB_PROJECT, name=run_name)
    epoch = 0
    global_step = 0

    while True:
        print(f"[Dendrites] Starting epoch {epoch}")
        model.train()

        for batch in train_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            optimizer.zero_grad()

            global_step += 1
            if global_step % 50 == 0:
                wandb.log({"train/loss": loss.item(), "step": global_step})


        train_metrics = evaluate(model, train_dataloader)
        train_acc = train_metrics["accuracy"]
        train_loss = train_metrics.get("loss", None)

        log_dict = {
            "epoch": epoch,
            "train/accuracy": train_acc,
        }
        if train_loss is not None:
            log_dict["train/epoch_loss"] = train_loss

        wandb.log(log_dict)

        # ----------------------------
        # Evaluate on validation set (orange line in the "good graph")
        # ----------------------------
        val_metrics = evaluate(model, eval_dataloader)
        val_loss = val_metrics["loss"]
        val_acc = val_metrics["accuracy"]

        print(f"[Dendrites] Epoch {epoch} | train_acc={train_acc:.4f} | val_loss={val_loss:.4f} | val_acc={val_acc:.4f}")

        wandb.log(
            {
                "val/loss": val_loss,
                "val/accuracy": val_acc,
                "epoch": epoch,
            }
        )

        # Optional extra score to PAI tracker (not required, but fine)
        GPA.pai_tracker.add_extra_score(val_acc, "Val Accuracy")

        model, restructured, training_complete = GPA.pai_tracker.add_validation_score(
            val_acc,
            model,
        )

        model.to(device)

        wandb.log({
            "pai/restructured": int(restructured),
            "pai/training_complete": int(training_complete),
            "pai/params": sum(p.numel() for p in model.parameters()),
            "epoch": epoch,
        })

        if training_complete:
            print("PerforatedAI reports training is complete. Stopping.")
            break

        if restructured:
            print("Model structure changed (dendrites added/absorbed). Rebuilding optimizer & scheduler.")
            optimizer, _ = GPA.pai_tracker.setup_optimizer(
                model,
                optim_args,
                sched_args,
            )

        epoch += 1
        if epoch >= max_epochs:
            print("Hit safety max_epochs limit. Stopping.")
            break

    run.finish()
    return model


dendritic_model = train_with_dendrites(
    dendritic_model,
    dend_optimizer,
    optim_args,
    sched_args,
    max_epochs=20,
)

# === FINAL EVAL ===
dend_metrics = evaluate(dendritic_model, eval_dataloader)
print("Compressed + dendrites metrics:", dend_metrics)
print("Compressed + dendrites params:", count_parameters(dendritic_model))

[Dendrites] Starting epoch 0
[Dendrites] Epoch 0 | train_acc=0.8504 | val_loss=0.4455 | val_acc=0.7901
Adding validation score 0.79013761
Checking PAI switch with mode n, switch mode DOING_SWITCH_EVERY_TIME, epoch 0, last improved epoch 0, total epochs 0, n: 10, num_cycles: 0
Returning True - switching every time
Importing best Model for switch to PA...
Switching back to N...
Resetting committed to initial rate to False
Saving model before starting normal training to retain PBNodes regardless of next N Phase results
Model structure changed (dendrites added/absorbed). Rebuilding optimizer & scheduler.
[Dendrites] Starting epoch 1
[Dendrites] Epoch 1 | train_acc=0.8897 | val_loss=0.4324 | val_acc=0.8073
Adding validation score 0.80733945
Checking PAI switch with mode n, switch mode DOING_SWITCH_EVERY_TIME, epoch 1, last improved epoch 1, total epochs 1, n: 10, num_cycles: 2
Returning True - switching every time
Importing best Model for switch to PA...
Switching back to N...
Resetting com

epoch,▁▁▁▃▃▃▆▆▆███
pai/params,▁▄██
pai/restructured,███▁
pai/training_complete,▁▁▁█
step,▁▁▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇████
train/accuracy,▁▅▇█
train/epoch_loss,█▄▂▁
train/loss,█▇▅▆▅▆▃▃▄▄▄▄▅▃▅▄▃▂▅▂▄▄▃▃▅▂▃▃▂▃▃▁▄▂▃▂▃▂▃▂
val/accuracy,▁▄▆█
val/loss,█▂▁▇
epoch,3


Compressed + dendrites metrics: {'loss': 0.44427441921802835, 'accuracy': 0.8314220183486238}
Compressed + dendrites params: 6062112


In [18]:
print("\n=== Final Comparison ===")

print("Baseline:")
print("  Accuracy:", baseline_metrics["accuracy"])
print("  Params:", count_parameters(baseline_model))
print("  Batch time (10 batches):", benchmark_inference(baseline_model, eval_dataloader))

print("\nCompressed (no dendrites):")
print("  Accuracy:", compressed_metrics["accuracy"])
print("  Params:", count_parameters(compressed_model))
print("  Batch time (10 batches):", benchmark_inference(compressed_model, eval_dataloader))

print("\nCompressed + dendrites:")
print("  Accuracy:", dend_metrics["accuracy"])
print("  Params:", count_parameters(dendritic_model))
print("  Batch time (10 batches):", benchmark_inference(dendritic_model, eval_dataloader))


=== Final Comparison ===
Baseline:
  Accuracy: 0.9025229357798165
  Params: 66955010
  Batch time (10 batches): 0.02549612522125244

Compressed (no dendrites):
  Accuracy: 0.7912844036697247
  Params: 4386178
  Batch time (10 batches): 0.005823183059692383

Compressed + dendrites:
  Accuracy: 0.8314220183486238
  Params: 6062112
  Batch time (10 batches): 0.019985342025756837
